In [4]:
import os
import sys
import numpy as np

import keras

from keras_pipeline.models import InceptionV3
from keras_pipeline.models import RetinaNet, RetinaNetConfig


In [5]:
model_config = RetinaNetConfig(num_classes = 7, input_shape=(800, 800, 3))
model = RetinaNet(model_config)
model.output_shape

[(None, 300, 4), (None, 300), (None, 300)]

In [3]:
"""Get Model Anchor based on input_shape"""
def get_model_anchors():
    x = np.zeros((1,) + model.input_shape[1:])
    anchor_model = keras.models.Model(
        inputs  = model.input,
        outputs = model.get_layer('anchors').output
    )
    
    return anchor_model.predict(x)

model_anchors = get_model_anchors()
model_anchors.shape

(1, 111798, 4)

In [8]:
from keras_pipeline.utils import anchors

gen_anchors = anchors.all_anchors_for_shape((800, 800, 3), model_config.get_feature_pyramid_shapes())
gen_anchors.shape

(111798, 4)

In [11]:
np.sum(model_anchors[0] - gen_anchors) / np.prod(gen_anchors.shape)

2.939809117011683e-07

In [7]:
model_config.compute_pyramid_feautre_shapes_for_img_shape((800, 800))

(array([97., 97.]),
 array([48., 48.]),
 array([23., 23.]),
 array([12., 12.]),
 array([6., 6.]))

In [22]:
i = -4
model.layers[i].name, model.layers[i].output_shape

AttributeError: The layer "classification_submodel has multiple inbound nodes, with different output shapes. Hence the notion of "output shape" is ill-defined for the layer. Use `get_output_shape_at(node_index)` instead.